In [75]:
import numpy as np
from finn.util.visualization import showInNetron
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.general import *
import copy

model = ModelWrapper("/tmp/quartznet.onnx")
model = model.transform(GiveUniqueNodeNames())

model.save("/tmp/quartznet_uniqueNames.onnx")
#showInNetron("/tmp/quartznet_uniqueNames.onnx")

list_of_repetitions = []
for idx, n in enumerate(model.graph.node):
    is_fork = model.is_fork_node(n)
    if is_fork is True:
        rep = {}
        start_node_id = idx
        rep[n.name] = idx
        for idx, n in enumerate(model.graph.node):
            if idx<=start_node_id:
                continue
            is_join = model.is_join_node(n)
            if is_join is True:
                end_node_id = idx+1 # one node after the join node
                end_node_name = model.graph.node[end_node_id].name
                rep[end_node_name] = end_node_id
                list_of_repetitions.append(rep)
                break
        
print("{}".format(list_of_repetitions))

rep_structure = list_of_repetitions[11] #0-14
start_end = []
for v in rep_structure.values():
    start_end.append(v)
print(start_end)
start_node_id = start_end[0]
end_node_id = start_end[1]

nodes = copy.deepcopy(model.graph.node)
for idx, n in enumerate(nodes):
    if idx<start_node_id or idx>end_node_id:
        model.graph.node.remove(n)

model.transform(RemoveUnusedTensors())
        
model.save("/tmp/quartznet_repetitive_nodes.onnx")
#showInNetron("/tmp/quartznet_repetitive_nodes.onnx")



[{'Mul_3': 9, 'MultiThreshold_13': 68}, {'Mul_26': 69, 'MultiThreshold_25': 128}, {'Mul_49': 129, 'MultiThreshold_37': 188}, {'Mul_72': 189, 'MultiThreshold_49': 248}, {'Mul_95': 249, 'MultiThreshold_61': 308}, {'Mul_118': 309, 'MultiThreshold_73': 368}, {'Mul_141': 369, 'MultiThreshold_85': 428}, {'Mul_164': 429, 'MultiThreshold_97': 488}, {'Mul_187': 489, 'MultiThreshold_109': 548}, {'Mul_210': 549, 'MultiThreshold_121': 608}, {'Mul_233': 609, 'MultiThreshold_133': 668}, {'Mul_256': 669, 'MultiThreshold_145': 728}, {'Mul_279': 729, 'MultiThreshold_157': 788}, {'Mul_302': 789, 'MultiThreshold_169': 848}, {'Mul_325': 849, 'MultiThreshold_181': 908}]
[669, 728]


In [76]:
from onnx import TensorProto
from onnx import helper as oh
import onnx
from finn.transformation.general import GiveUniqueParameterTensors

model = ModelWrapper("/tmp/quartznet_repetitive_nodes.onnx")

for n in model.graph.node:
    in_tensor_name = n.input[0]
    in_tensor_shape = model.get_tensor_shape(in_tensor_name)
    break

in_tensor_name = model.graph.node[0].input[0]
in_tensor_shape = model.get_tensor_shape(in_tensor_name)

out_tensor_name =  model.graph.node[-1].input[0]
out_tensor_shape = model.get_tensor_shape(out_tensor_name)

#in_tensor=onnx.helper.make_tensor_value_info("in_tensor", onnx.TensorProto.FLOAT, [1, 256, 128])
#out_tensor=onnx.helper.make_tensor_value_info("out_tensor", onnx.TensorProto.FLOAT, [1, 256, 128])
in_tensor=onnx.helper.make_tensor_value_info("in_tensor", onnx.TensorProto.FLOAT, in_tensor_shape)
out_tensor=onnx.helper.make_tensor_value_info("out_tensor", onnx.TensorProto.FLOAT, out_tensor_shape)

old_input = model.graph.input[0]
model.graph.input.remove(old_input)
model.graph.input.extend([in_tensor])

old_output = model.graph.output[0]
model.graph.output.remove(old_output)
model.graph.output.extend([out_tensor])

input_node = model.graph.node[0]
input_node.input[0] = 'in_tensor'

output_node = model.graph.node[-1]
output_node.output[0] = 'out_tensor'

##### To have unique initializers:
model = model.transform(GiveUniqueParameterTensors())
#list_of_initializers = []
#for n in model.graph.node:
#    init_name = n.input[1]
#    if init_name in list_of_initializers:
#        init_val = model.get_initializer(n.input[1])
#        init_shape = np.shape(init_val)
#        if len(init_shape) < 1:
#            init_shape = (1,)
        
#        new_init = oh.make_tensor_value_info(
#            model.make_new_valueinfo_name(), TensorProto.FLOAT, init_shape
#        )
#        model.graph.value_info.append(new_init)
#        model.set_initializer(new_init.name, init_val)
#        n.input[1] = new_init.name
#    else:
#        list_of_initializers.append(n.input[1])
#####

from finn.transformation.general import GiveRandomTensorNames, GiveReadableTensorNames
model = model.transform(GiveRandomTensorNames())
model = model.transform(GiveReadableTensorNames())

model.save("/tmp/quartznet_repetitive_nodes_subpart.onnx")
showInNetron("/tmp/quartznet_repetitive_nodes_subpart.onnx")



Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_repetitive_nodes_subpart.onnx' at http://0.0.0.0:8081


In [77]:
from finn.transformation.streamline import *
from finn.transformation.streamline.reorder import MoveMulPastDWConv, MoveLinearPastEltwiseAdd, MoveMulPastFork
from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.streamline.absorb import AbsorbConsecutiveTransposes # No effect (only on consecutive transpose nodes)
from finn.transformation.streamline.absorb import AbsorbTransposeIntoMultiThreshold
from finn.util.basic import get_by_name
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
from finn.transformation.general import GiveRandomTensorNames, GiveReadableTensorNames

model = ModelWrapper("/tmp/quartznet_repetitive_nodes_subpart.onnx")

model = model.transform(GiveRandomTensorNames())
model = model.transform(GiveReadableTensorNames())

# Convert to supported format
model = model.transform(Change3DTo4DTensors())

# Collapse BatchNorm to Add and Mul
model = model.transform(BatchNormToAffine())

# Group additions
model = model.transform(MoveAddPastMul())
model = model.transform(MoveAddPastConv())
model = model.transform(MoveAddPastMul())

# Group multiplications
#### Move mul past fork
model = model.transform(MoveMulPastFork())
model = model.transform(MoveScalarMulPastConv())
model = model.transform(MoveMulPastDWConv())

# Move Mul/Add past join node
model = model.transform(MoveLinearPastEltwiseAdd())

# Collapes additions & multiplications
model = model.transform(CollapseRepeatedAdd())
model = model.transform(CollapseRepeatedMul())

# Absorb Add/Mul into multithreshold
model = model.transform(AbsorbAddIntoMultiThreshold())
model = model.transform(FactorOutMulSignMagnitude())
model = model.transform(Absorb1BitMulIntoConv())
model = model.transform(AbsorbMulIntoMultiThreshold())

# Ensure thresholds are integers
## Add quantization annotation to ensure RoundAndClipThresholds works
from finn.transformation.infer_datatypes import InferDataTypes
model = model.transform(InferDataTypes())
        
model = model.transform(RoundAndClipThresholds())

model = model.transform(LowerConvsToMatMul())

model = model.transform(AbsorbTransposeIntoMultiThreshold())


model.save("/tmp/quartznet_repetitive_nodes_subpart_streamlined.onnx")
showInNetron("/tmp/quartznet_repetitive_nodes_subpart_streamlined.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_repetitive_nodes_subpart_streamlined.onnx' at http://0.0.0.0:8081


In [78]:
import finn.core.onnx_exec as oxe
from finn.util.basic import *

def generate_random_input(model):
    """ Creates input dictionary with a random numpy array that matches the input tensor shape """
    i_shape = []
    input_dict={}
    for i in range(len(model.graph.input)):
        input_node = model.graph.input[i]
        input_node_name = input_node.name
        input_node_shape = model.get_tensor_shape(input_node_name)

        #i_val = gen_finn_dt_tensor(DataType.FLOAT32, input_node_shape)
        i_val = np.random.randint(0, 128, input_node_shape).astype(np.float32) #float32 is expected input
        
        input_dict[input_node_name] = i_val
    return input_dict

model = ModelWrapper("/tmp/quartznet_repetitive_nodes_subpart.onnx")
input_dict = generate_random_input(model)
print(input_dict)
output_node_name = model.graph.output[0].name
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
expected_old = output_dict[output_node_name]

print("{}\n{}".format(np.shape(expected_old),expected_old))

{'global_in': array([[[115.,  93., 107., ..., 120.,  67.,   8.],
        [ 34., 127.,  23., ..., 126.,  64.,  34.],
        [124.,   7.,  90., ...,  53.,  30.,  66.],
        ...,
        [ 63.,  24., 124., ...,  94.,  81.,  23.],
        [ 62.,  70.,  67., ...,  18.,  16.,  82.],
        [ 53.,  51.,  49., ...,   2.,  54.,  83.]]], dtype=float32)}
(1, 512, 128)
[[[ 0.  0.  0. ...  0.  0.  0.]
  [15. 15. 15. ... 15. 15. 15.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]]


In [79]:
old_input_name = model.graph.input[0].name
old_input_dict_val = input_dict[old_input_name]

model = ModelWrapper("/tmp/quartznet_repetitive_nodes_subpart_streamlined.onnx")

input_dict_val = old_input_dict_val
if len(np.shape(input_dict_val))<4:
    input_dict_val = np.reshape(input_dict_val, np.shape(input_dict_val)+(1,))
    
input_dict[model.graph.input[0].name] = input_dict_val

assert (input_dict_val==np.reshape(old_input_dict_val,np.shape(input_dict_val))).all()

output_node_name = model.graph.output[0].name
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
expected_new = output_dict[output_node_name]

expected_new = np.reshape(expected_new, np.shape(expected_old))

print("{}\n{}".format(np.shape(expected_new),expected_new))

assert(expected_old==expected_new).all()



(1, 512, 128)
[[[ 0.  0.  0. ...  0.  0.  0.]
  [15. 15. 15. ... 15. 15. 15.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]]
